# Requirments

- pandas
- numpy
- scikit-learn

Please run the cell below if they are not installed.

In [150]:
%pip install pandas numpy scikit-learn

10370.94s - pydevd: Sending message related to process being replaced timed-out after 5 seconds

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


The cell below consists all the `import` statements used. No other libraries are imported further down below.

In [151]:
import pandas as pd
import numpy as np
import random
import math

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

# Import Data

The cell below imports the information about the columns if they are _Continuous_ or _Categorical_.

In [152]:
type_df = pd.read_csv('type_info.csv')
columns = list(type_df['name'])
columns.reverse()

Train set and test set are imported.

The variables declared here are important to the overall notebook.

In [153]:
train_df = pd.read_csv('training.data', names=columns)
test_df = pd.read_csv('test.data', names=columns)

## Cleaning the imported data.

- Replacing '?' with median for continuous attributes. And middle value for categorical attributes.
- Converting values in columns to the appropriate data type.

In [154]:
for column in list(type_df[type_df['type']=='Continuous']['name']):
  train_df[column] = train_df[column].replace('?', np.nan)
  test_df[column] = test_df[column].replace('?', np.nan)
  if(train_df[column].dtype == 'object'):
    train_df[column] = train_df[column].astype('float64')
  if(test_df[column].dtype == 'object'):
    test_df[column] = test_df[column].astype('float64')
  median_train = train_df[column].median()
  train_df[column] = train_df[column].fillna(median_train)
  test_df[column] = test_df[column].fillna(median_train)


In [155]:
for column in list(type_df[type_df['type']=='Categorical']['name']):
  unique_train = train_df[column].unique()
  unique_train = np.delete(unique_train, np.where(unique_train == '?'))
  unique_train.sort()
  median_train = unique_train[int(len(unique_train)/2)]
  unique_test = test_df[column].unique()
  unique_test = np.delete(unique_test, np.where(unique_test == '?'))
  unique_test.sort()
  train_df[column] = train_df[column].replace('?', np.nan)
  test_df[column] = test_df[column].replace('?', np.nan)
  train_df[column] = train_df[column].fillna(median_train)
  test_df[column] = test_df[column].fillna(median_train)

# Decision Tree Classifier
The following class contains all the functions required to calculate `entropy`, `gain`, `gain ratio,` `gini`, `gini index`, appropriately split continuous attributes, compute `f1 score`, perform cross validation. It also includes `get importance` funtion which is required for building a decision tree. `predict` and `fit` functions are also implemented for easier access.

In [156]:
class DecisionTreeClassifier():

  def __init__(self, train, features, target):
      self.__train = train
      self.__features = features
      self.__target = target
      self.__tree = {}
      self.__test_pred = []
      self.__f1_scores = {}
      

  def get_agg_f1_scores(self):
    if self.__tree == {}:
      raise Exception('Please run fit or cross_validate')
    if self.__f1_scores == {}:
      raise Exception('Please run cross_validate')
    return self.__f1_scores
  
  def get_tree(self):
    if self.__tree == {}:
      raise Exception('Please run fit or cross_validate')
    return self.__tree
  
  def fit(self, train, features, target, solver='c4.5'):
    self.__train = train
    self.__features = features
    self.__target = target
    self.__solver = solver
    self.__tree = self.__decision_tree_learning(self.__train, self.__features, self.__train)

  def __calculate_entropy(self, attribute_column):
    total_count = len(attribute_column)
    counts = attribute_column.value_counts()
    goals = {}
    for item in attribute_column.unique():
      goals[item] = counts.at[item]/total_count
    entropy_value = 0.0
    for goal in goals:
      entropy_value -= goals[goal] * math.log(goals[goal], 2)
    return entropy_value

  def __calculate_gain_ratio(self, train):
    total_entropy = self.__calculate_entropy(train[self.__target])
    gain = {}
    iv = {}
    gain_ratio = {}
    best_split_gain = {}
    best_split_gain_ratio = {}
    for feature in self.__features:
      if feature in list(type_df[type_df['type'] == 'Continuous']['name']):
        best_split_gain[feature], gain[feature], best_split_gain_ratio[feature], gain_ratio[feature], _, _ = self.__split_continuous(train, feature)
      else:
        attr_unique = train[feature].unique()
        gain_att = 0
        iv[feature] = 0
        for item in attr_unique:
          d_ratio = train[feature].value_counts().at[item]/len(train[feature])
          gain_att += d_ratio * (self.__calculate_entropy(train[self.__target][train[feature]==item]))
          iv[feature] -= d_ratio * math.log(d_ratio, 2)
        gain[feature] = total_entropy - gain_att
        if(iv[feature] == 0):
          gain_ratio[feature] = 0
        else:
          gain_ratio[feature] = gain[feature]/iv[feature]
    return best_split_gain, gain, best_split_gain_ratio, gain_ratio

  def __calculate_gini(self, attribute_column):
    total_count = len(attribute_column)
    counts = attribute_column.value_counts()
    goals = {}
    for item in attribute_column.unique():
      goals[item] = counts.at[item]/total_count

    gini = 1.0
    for goal in goals:
      gini -= goals[goal] * goals[goal]
    return gini


  def __calculate_gini_index(self, train):
    gini_index = {}
    best_split_gini_index = {}
    for feature in self.__features:
      if feature in list(type_df[type_df['type'] == 'Continuous']['name']):
        _, _, _, _, best_split_gini_index[feature], gini_index[feature] = self.__split_continuous(train, feature)
      else:
        attr_unique = train[feature].unique()
        gini_att = 0
        for item in attr_unique:
          gini_att += train[feature].value_counts().at[item]/len(train[feature]) * (self.__calculate_gini(train[self.__target][train[feature]==item]))
        gini_index[feature] = gini_att
    return best_split_gini_index, gini_index


  def __get_importance(self, train, attributes):
    best_split_gain, gain_calc, best_split_gain_ratio, gain_ratio_calc = self.__calculate_gain_ratio(train)
    best_split_gini_index, gini_index_calc = self.__calculate_gini_index(train)
    attributes.sort()
    gain_ratio = {}
    gini_index = {}
    gain = {}
    for key in attributes:
      gain_ratio[key] = gain_ratio_calc[key]
      gini_index[key] = gini_index_calc[key]
      gain[key] = gain_calc[key]
    if(self.__solver=='c4.5'):
      return best_split_gain_ratio, max(gain_ratio, key=gain_ratio.get)
    elif(self.__solver=='cart'):
      return best_split_gini_index, min(gini_index, key=gini_index.get)
    elif(self.__solver=='id3'):
      return best_split_gain, max(gain, key=gain.get)
    else:
      return None

  def __split_continuous(self, train, attribute):
    sorted_attribute = list(train[attribute].sort_values())
    splits = []
    for i in range(0, len(sorted_attribute)-1):
      splits.append((sorted_attribute[i] + sorted_attribute[i+1])/2)
    # print(splits)
    split_gain = {}
    split_iv = {}
    split_gain_ratio = {}
    split_gini_index = {}
    for split in splits:
      d_less = train[train[attribute] <= split][attribute].count()
      d_more = train[train[attribute] > split][attribute].count()
      split_gain[split] = self.__calculate_entropy(train[self.__target]) - ((d_less * self.__calculate_entropy(train[self.__target][train[attribute] <= split]) + d_more * self.__calculate_entropy(train[self.__target][train[attribute] > split]))/len(train))
      if(d_less == 0):
        d_less = len(train)
      if(d_more == 0):
        d_more = len(train)
      split_iv[split] = - ((d_less/len(train)) * math.log(d_less/len(train), 2) + (d_more/len(train)) * math.log(d_more/len(train), 2))
      if(split_iv[split] == 0):
        split_gain_ratio[split] = 0
      else:
        split_gain_ratio[split] = split_gain[split]/split_iv[split]
      split_gini_index[split] = (d_less * self.__calculate_gini(train[self.__target][train[attribute] <= split]) + d_more * self.__calculate_gini(train[self.__target][train[attribute] > split]))/len(train)
    max_gain_key = max(split_gain, key=split_gain.get)
    max_gain_ratio_key = max(split_gain_ratio, key=split_gain_ratio.get)
    min_gini_index_key = min(split_gini_index, key=split_gini_index.get)
    return max_gain_key, split_gain[max_gain_key], max_gain_ratio_key, split_gain_ratio[max_gain_ratio_key], min_gini_index_key, split_gini_index[min_gini_index_key]

  def get_f1_score(self, test):
    if self.__tree == {}:
      raise Exception('Please run fit or cross_validate')
    test_features = test.drop('A16', axis=1)
    y_pred = self.predict(test_features)
    y_true = list(test['A16'])
    return f1_score(y_true, y_pred, pos_label='+')

  def cross_validate(self, train = train_df, splits=10, solver='c4.5'):
    self.__f1_scores[solver] = {}
    kf = KFold(n_splits=splits)
    kf.get_n_splits(train_df)
    for i, (train_index, test_index) in enumerate(kf.split(train_df)):
        train_set = train.iloc[train_index]
        test_set = train.iloc[test_index]
        self.fit(train_set, self.__features, self.__target, solver=solver)
        self.__f1_scores[solver][i] = self.get_f1_score(test_set)
    return self.__f1_scores[solver]

  def predict(self, test):
    self.__test_pred = []
    test_dict = test.to_dict(orient='index')
    for index, row in test_dict.items():
      self.__classify(row, self.__tree)
    return self.__test_pred

  def __classify(self, row, tree):
    key = list(tree)[0]
    value = tree[key]
    if(key in list(type_df[type_df['type'] == 'Continuous']['name'])):
      for item in list(value):
        if(eval(str(row[key])+str(item))):
          branch = item
          if type(value[branch]) is not dict:
            self.__test_pred.append(value[branch])
          else:
            self.__classify(row, value[branch])
    else:
      branch = row[key]
      key_list = list(value)
      if branch in key_list:
        branch = branch
      else:
        branch = random.choice(key_list)
      if type(value[branch]) is not dict:
          self.__test_pred.append(value[branch])
      else:
        self.__classify(row, value[branch])

  def __decision_tree_learning(self, examples, attributes, parent_examples):
    tree = {}
    if len(examples[self.__target]) == 0:
      return parent_examples[self.__target].value_counts().index[0]
    elif len(examples[self.__target].unique()) == 1:
      return examples[self.__target].unique()[0]
    elif len(attributes) == 0:
      return examples[self.__target].value_counts().index[0]
    else:
      best_split, best_attr = self.__get_importance(examples, attributes)
      tree[best_attr] = {}
      if best_attr in list(type_df[type_df['type'] == 'Continuous']['name']):
        for i in ['<=' , '>']:
          exs = examples.query(f'{best_attr} {i} {best_split[best_attr]}')
          subtree = self.__decision_tree_learning(exs, list(set(attributes)-set([best_attr])), examples)
          tree[best_attr][f'{i} {str(best_split[best_attr])}'] = subtree
      else:
        unique_train_best_attr = self.__train[best_attr].unique()
        unique_train_best_attr.sort()
        for att_val in unique_train_best_attr:
          exs = examples[examples[best_attr] == att_val]
          subtree = self.__decision_tree_learning(exs, list(set(attributes)-set([best_attr])), examples)
          tree[best_attr][att_val] = subtree
      return tree

Initialization of `DecisionTreeClassifier` class takes training data set (`train_df`) which was imported and cleaned earlier. And it also takes a list of `feature` attributes, and the name of the `target` attribute.

In [157]:
train_columns = list(train_df.columns)
train_target = 'A16'
train_features = train_columns.copy()
train_features.remove(train_target)

dt = DecisionTreeClassifier(train_df, train_features, train_target)

The following cell performs cross-validation with 10 splits. `solvers` list can be modified to include solvers that are required. Currently it performs cross validation on `id3`, `c4.5`, and `cart` and stores them in a class variable. The solver names are case-sensitive. `cross_validate` function calls `fit` function internally. It can also be called separately.

In [158]:
solvers = ['id3', 'c4.5', 'cart']
for solver in solvers:
    dt.cross_validate(solver=solver)

Access the `f1_score` values computed earlier for all the `solvers`.

In [159]:
agg_f1_scores = dt.get_agg_f1_scores()

In [160]:
agg_f1_scores

{'id3': {0: 0.7368421052631579,
  1: 0.8979591836734694,
  2: 0.64,
  3: 0.75,
  4: 0.6829268292682927,
  5: 0.8333333333333334,
  6: 0.7636363636363637,
  7: 0.7906976744186046,
  8: 0.7307692307692307,
  9: 0.75},
 'c4.5': {0: 0.7692307692307693,
  1: 0.8461538461538461,
  2: 0.84,
  3: 0.8695652173913043,
  4: 0.6976744186046512,
  5: 0.8461538461538461,
  6: 0.8,
  7: 0.8,
  8: 0.8363636363636363,
  9: 0.7857142857142857},
 'cart': {0: 0.7368421052631579,
  1: 0.7391304347826086,
  2: 0.7659574468085106,
  3: 0.6976744186046512,
  4: 0.8085106382978723,
  5: 0.7843137254901961,
  6: 0.7547169811320755,
  7: 0.8,
  8: 0.72,
  9: 0.7037037037037037}}

We choose the best split where we have the highest `f1_score` and call `fit` function on that specific train split. `f1_score` values are calculated on test set (`test_df`) provided and printed as the cell output. Currently it performs prediction for all the solvers present in `agg_f1_scores` variable. That can be adjusted by adjusting the `solvers` in the cell where the `cross_validate` function was called.

In [161]:
kf_best = KFold(n_splits=10)
for solver in list(agg_f1_scores):
    best_train_indices = []
    best_split = max(agg_f1_scores[solver], key=agg_f1_scores[solver].get)
    for i, (train_index, _) in enumerate(kf_best.split(train_df)):
        if i == best_split:
            train_set = train_df.iloc[train_index]
    dt.fit(train_set, train_features, train_target, solver)
    print(f'F1 Score for {solver}: {dt.get_f1_score(test_df)}')

F1 Score for id3: 0.8062015503875969
F1 Score for c4.5: 0.8
F1 Score for cart: 0.784
